# Collaborative Filtering
This notebook will contain the steps to create a collaborative fitering recommendation system for the anime and ratings datasets.

In [1]:
import pandas as pd
import numpy as np

## Preprocessing the Data
### Anime Dataset
Let's read the anime dataset into a pandas dataframe

In [3]:
anime_df = pd.read_csv("datasets/cleaned_anime.csv")
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64.0,9.26,793665
1,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.25,114262
2,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24.0,9.17,673572
3,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.16,151266
4,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10.0,9.15,93351


With collaborative filtering, we actually don't need a lot of the animes' metadata as the recommendations are primarily dictated by the users who have watched a certain anime and not the actual contents. This contrasts with content based filtering which uses the anime information to predict what a user would like. As a result of this, we can remove a lot of information that we don't need from the anime data both to make the process clearer and save memory.

In [4]:
# Remove information we don't need
anime_df = anime_df.loc[:, ["anime_id", "name", "rating"]]
anime_df.head()

,anime_id,name,rating
0,5114,Fullmetal Alchemist: Brotherhood,9.26
1,28977,Gintama°,9.25
2,9253,Steins;Gate,9.17
3,9969,Gintama&#039;,9.16
4,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,9.15


### Ratings Dataset
Read in the data into a dataframe

In [5]:
rating_df = pd.read_csv("datasets/cleaned_rating.csv")
rating_df.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


The only thing to do for the ratings data is to remove missing values

In [6]:
# Remove missing values
rating_df.dropna(inplace=True)
# How many missing values do we have?
rating_df.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

# Building the Recommendation System
To build the collaborative filtering system, I will be using a sample user input from animes I have watched in the past. To build it, I will be using the SciKit Surprise library.

In [17]:
%%time
# Import the surprise library
from surprise import Reader
from surprise import Dataset

# reader parses the file containing the ratings
# Our rating scale is from 1 to 10 inclusive
reader = Reader(rating_scale=(1, 10))

# Load the dataframe into the model's dataset
data = Dataset.load_from_df(rating_df[["user_id", "anime_id", "rating"]], reader)
data

CPU times: user 1.91 s, sys: 175 ms, total: 2.09 s
Wall time: 2.26 s


Now that we have our rating data loaded into surprise, we can create our collaborative filtering system. For this, I decided to use the famous SVD algorithm that is included with surprise. I will also be using surprise's GridSearchCV which is very similar to sklearn's GridSearchCV to find the best parameters for the best accuracy.

In [8]:
# Import the SVD algorithm
from surprise import SVD

In [9]:
# Import GridSearchCV to tune parameters
from surprise.model_selection import GridSearchCV

In [19]:
%%time
# This cell took a LONG time to run on my computer
# Don't suggest running it unless you have time to waste



# Create parameters combinations
params = {
    "n_epochs": [10, 15], "lr_all": [0.003, 0.005, 0.007], "reg_all": [0.01, 0.02, 0.03]
}

# Run the grid search using SVD and the parameters to find the best parameters for Root Mean Square Error and Mean Absolute Error
gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=3, joblib_verbose=2, n_jobs=-2)

gs.fit(data)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.


KeyboardInterrupt: 

In [20]:
# Best RMSE score
print(gs.best_score["rmse"])

# Best combination of parameters for the best RMSE
print(gs.best_params["rmse"])

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

Now that we have the best parameters from the GridSearchCV we ran, we can now set our recommendation algorithm to the parameters

In [15]:
%%time
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

NameError: name 'gs' is not defined

This is as far as my Google searches and tutorials got me. What I suspect is that using `algo.predict(user_id, anime_id).est` I can get the estimated rating that the user would give for that specific anime. Perhaps if I made a numpy array that stored the estimated ratings for a number of animes for a given user, I could find the ones with the highest estimated ratings and recommend those.

In [46]:
# Just testing out the prediction method on user 4271 on anime id 7088
pred = algo.predict(4271, 7088).est
pred

8.01538233688656

Last thing to do is to export the recommendation system into a Pickle file for potentially usage in the future.

In [48]:
# Import the dump function
from surprise.dump import dump
# Create the Pickle file for the SVD algorithm
dump("recommender.pkl", algo)